<a href="https://colab.research.google.com/github/siliconemonster/IA-2020.1/blob/main/Algoritmo_Genetico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<h1> ALGORITMO GENÉTICO**


Aline Freire de Rezende - 116110571<br>
Larissa Galeno - 116

In [2]:
import numpy as np
import random

<h2>1 - Modelagem do Indivíduo

<h3>a) Utilize a mesma representação de tabuleiro usada na tarefa do Hill Climbing.

No caso de um tabuleiro 4x4, como o caso acima, temos um vetor de 1x4, vazio. Só pode haver uma rainha por coluna, então a coluna em que existir uma, será representada pelo número da linha.<br>
**Por exemplo:**<br>
Um tabuleiro do tipo: <br>
[0 &nbsp; 0 &nbsp; 0 &nbsp; 0]<br>
[R &nbsp; 0 &nbsp; 0 &nbsp; R]<br>
[0 &nbsp; R &nbsp; 0 &nbsp; 0]<br>
[0 &nbsp; 0 &nbsp; R &nbsp; 0]<br>
onde 0 representa onde está vazio, e R representa onde há uma rainha; na modelagem em questão, é representado por: <br>
[2 &nbsp; 3 &nbsp; 4 &nbsp; 2]

<h3>b) Proponha uma representação binária para o tabuleiro.

<h2>2 - Geração População Inicial

<h3>a) Defina uma função que dado o tamanho n de uma população, gera aleatoriamente um conjunto de n indivíduos. A função deve funcionar para os dois tipos de indivíduos definidos no item anterior.

In [9]:
def geraTabuleiro(n):
  tabuleiro = []

  for x in range(n):
    posicaoRainha = random.randint(1, n)
    tabuleiro.append(posicaoRainha)

  return tabuleiro

geraTabuleiro(4)

[4, 3, 4, 1]

In [12]:
def geraPopulacao(numIndividuos, n):
  populacao = []

  for i in range(numIndividuos):
    populacao.append(geraTabuleiro(n)) ##### FALTA LIDAR COM ELEMENTOS REPETIDOS

  return populacao

geraPopulacao(10, 4)

[[2, 4, 1, 1],
 [1, 3, 1, 1],
 [1, 2, 1, 1],
 [3, 3, 1, 4],
 [1, 2, 4, 1],
 [2, 4, 1, 1],
 [4, 1, 2, 3],
 [4, 1, 2, 2],
 [1, 4, 4, 4],
 [1, 1, 1, 4]]

<h2>3 - Operadores: Para os dois tipos de indivíduos definidos acima:

<h3>a) Defina a função de adaptação usada para avaliar um tabuleiro T.

In [11]:
def avaliaAtaques(tabuleiro):
  n = len(tabuleiro)
  ataques = 0

  for i in range(n-1): # caso esteja na mesma coluna
    for j in range(i+1, n):
      if (tabuleiro[i] == tabuleiro[j]):
        ataques = ataques + 1

  for i in range(n-1): # caso esteja na mesma diagonal. Para baixo e para cima, respectivamente
    for j in range(i+1, n):
      if ((tabuleiro[i] == tabuleiro[j] + (j - i)) or (tabuleiro[i] == tabuleiro[j] - (j - i))):
        ataques = ataques + 1

  #print("Número de ataques:")
  #print(ataques)
  return ataques

tabuleiro = geraTabuleiro(4)
avaliaAtaques(tabuleiro)

2

<h3>b) Defina a função que dada uma população P, constrói a roleta viciada correspondente a P.


In [17]:
def roletaViciada(populacao):
  n = len(populacao)
  roleta = []

  for i in range(n):
    #print(populacao[i])
    r = avaliaAtaques(populacao[i])
    #print(r)
    roleta.append(r)
  
  total = sum(roleta)
  
  #print(roleta, total)
  return roleta, total

roletaViciada(geraPopulacao(10, 4))

([3, 5, 4, 3, 5, 2, 2, 6, 4, 3], 37)

<h3>c) Defina a função que dada uma população P constrói uma população intermediária correspondente a P. (seleção)


In [18]:
def selecao(populacao):
  #print("Populacao Inicial" + str(populacao))
  n = len(populacao)
  populacao_int = []

  roleta, total = roletaViciada(populacao)

  for i in range(n):
    sorteado = random.randint(0, total)
    #print("Sorteado " + str(sorteado))
    esquerda = 0
    direita = roleta[0]
    j = 0
    while(True):
      if sorteado == 0:
        populacao_int.append(populacao[j])
        break
      elif sorteado > esquerda and sorteado <= direita:
        populacao_int.append(populacao[j])
        break
      esquerda = direita
      direita = direita + roleta[j+1]
      j = j + 1
  
  return populacao_int

selecao(geraPopulacao(6, 4))

[[2, 4, 3, 3],
 [2, 1, 1, 2],
 [2, 3, 1, 4],
 [2, 2, 2, 3],
 [2, 1, 1, 2],
 [2, 1, 1, 2]]

<h3>d) Defina a função que faz o crossover entre dois indivíduos. (crossover)


In [13]:
def crossover(mae1, mae2):
  filhas = []
  n = len(mae1)
  pontoCorte = random.randint(0, n-2) #ponto corte
  print(pontoCorte)

  inicio1 = mae1[:pontoCorte+1]
  final1 = mae2[pontoCorte+1:]
  filha1 = inicio1 + final1
  filhas.append(filha1)

  inicio2 = mae2[:pontoCorte+1] 
  final2 = mae1[pontoCorte+1:]
  filha2 = inicio2 + final2
  filhas.append(filha2)


  return filhas

crossover([1, 2, 3, 3], [2, 3, 3, 4])

0


[[1, 3, 3, 4], [2, 2, 3, 3]]

<h3>e) Defina a função que faz a mutação em um dado indivíduo. (mutação)

In [15]:
#Troca o aleatorio escolhido por quem tá do lado direito, se o aleatorio for o ultimo troca com o primeiro
def mutacao(individuo):  #### EXISTEM OUTROS TIPOS DE MUTAÇÃO
  n = len(individuo)
  aleatorio = random.randint(0, n-1)
  #print(aleatorio)

  if (aleatorio == n-1):
    aux = individuo[aleatorio]
    individuo[aleatorio] = individuo[0]
    individuo[0] = aux

  else:
    aux = individuo[aleatorio]
    individuo[aleatorio] = individuo[aleatorio + 1]
    individuo[aleatorio + 1] = aux

  return individuo

mutacao([1,2,3,4,5,6,7,8])

[1, 2, 3, 4, 5, 7, 6, 8]

<h2>4 - Algoritmo Genético Básico

<h2>a) Implemente o algoritmo genético básico. Sua implementação deve ter como parâmetros de entrada:

* Tamanho da população
* Número de gerações
* Probabilidade de Crossover
* Probabilidade de Mutação
* Utilização de Elitismo: no elitismo, uma cópia do melhor indivíduo da geração Pi−1 é passada para a geração Pi, sem passar pelos operadores.


A saída do programa deve ser composta por 2 gráficos:

* geração × função de adaptação do melhor indivíduo da geração
* geração × média da função de adaptação dos indivíduos da geração

e o melhor indivíduo da última geração e o valor da sua função de adaptação.

In [8]:
def genetico(populacao, probCrossover, probMutacao):

  novaPopulacao = []

  while True:
    for i in range(len(populacao)):
      aux1 = random.randint(0, (len(populacao)))
      aux2 = random.randint(0, (len(populacao)))
      mae1 = populacao[aux1]
      mae2 = populacao[aux2] ##### FALTA GARANTIR QUE MÃES NÃO SÃO REPETIDAS
      
      probCalculada = random.uniform(0, probCrossover)
      if probCalculada < probCrossover:
        filha = crossover(mae1, mae2)
      
      probCalculada = random.uniform(0, probMutacao)
      if (probCalculada <= probMutacao):
        filha = mutacao(filha)
      novaPopulacao[i] = filha
    
    populacao = novaPopulacao # acho que isso está errado
    if (condicoesDeParada): # determinar condições de parada
      return melhorElemento # determinar quem é o melhor elemento

populacao = geraPopulacao(10, 4)
genetico(populacao)

TypeError: ignored

<h2>5 - Utilizando diferentes valores para os parâmetros de entrada, diferentes representações de indivíduos e diferentes tamanhos de tabuleiro (4,8,16,32), compare e discuta os resultados obtidos.